In [4]:
import pandas as pd
import numpy as np
import requests

In [5]:
api_key = 'RGAPI-1aa73176-1c31-48ce-951f-2cb80a23d37e'

In [6]:
#api_key = '발급받은 api'
sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' +'hideonbush' +'?api_key=' + api_key
r = requests.get(sohwan)
r.json()

{'id': 'wdWYyOJaVX32XmFMWfOCXSSU5F-zJ_OytC_Cki761MD-Qw',
 'accountId': 'YL60OrXJk2jJZ34Mc6_IMAviGMiH8mE7lED2YIh3IEzb',
 'puuid': 'J_kJBoZzbMptcWmyKwSvLl7qDWH1JjDohQSgkT42YitGY7CRuJj4KW5Dqv8yqi3jdLC27ZFJT8DlPA',
 'name': 'Hide on bush',
 'profileIconId': 6,
 'revisionDate': 1631444378000,
 'summonerLevel': 495}

In [7]:
tier_url = 'https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/' + r.json()['id'] +'?api_key=' + api_key
r2  = requests.get(tier_url)
r2.json()

[{'leagueId': 'd1ff3466-6c02-33b0-8085-9fa6a8b82dd5',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'CHALLENGER',
  'rank': 'I',
  'summonerId': 'wdWYyOJaVX32XmFMWfOCXSSU5F-zJ_OytC_Cki761MD-Qw',
  'summonerName': 'Hide on bush',
  'leaguePoints': 777,
  'wins': 687,
  'losses': 633,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [8]:
grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(grandmaster)#그마데이터 호출
r.json()
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True)#수집한 그마데이터 index정리
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
league_df = pd.concat([league_df, league_entries_df], axis=1) #열끼리 결합

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,GRANDMASTER,7YfCDpFYTYPUtpfzRPK4r2Dkm6Sf4X_pbAPHaOlNNJZNOZ...,zhi yu you xi,646,367,302,False,False,False,False
1,GRANDMASTER,SM3nUD5intVsoBaypSWismsmdaXXQSBqTw5my4a62Y2e18A,Roach,663,249,212,False,False,False,False
2,GRANDMASTER,nnMbIplr-rWpKZxW7_rbG1m3XPUJ9dC2L6dMeEL8qKRPRjE,Cherry Bloss0m,635,662,613,False,False,True,False
3,GRANDMASTER,JGdZHnFLU5Rd_CxvZPQ3wkH8_uPgOigvmyabqG1iEZr4SfY,cxybaby,656,213,143,False,False,False,False
4,GRANDMASTER,BE7xYGNZHue3Itk_e5K-yC6G8myFSBzVgi9irVpECdqNwR5Q,평범한 숟가락,602,841,798,True,False,False,True


In [ ]:
for i in range(len(league_df)):
    try:
        sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(sohwan)
        
        while r.status_code == 429:
            time.sleep(5)
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)
            
        account_id = r.json()['accountId']
        league_df.iloc[i, -1] = account_id
    
    except:
        pass


In [38]:
league_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,GRANDMASTER,tqJ56N1K0Su6MQskTC5XnBifGuE9xfZklh-LcXMa8ePQ7zM,Clever7,514,529,490,True,False,False,Jtd8ASj1Y11YPTzNq04YRoJVRNXOfrCbNRJ4NxIX0QLMVl8
1,GRANDMASTER,2Zmm3t8j3XaQRCSmdmRl-z4bYbS8avj644_rqAHDTIlG454,NEED HER,683,429,376,True,False,False,c3nU24-qwhbEsJnGe6_F4hUFTFEIMLMVBxy97pc7NHhVKJ0
2,GRANDMASTER,XjYDzbugUK3zi7YA__F541ltfQ_VMPTMF7i_BbWKR73ILA,Naehyun,720,709,673,True,False,False,H77sh4HmyGoW4eJRGuIVpGg4VsrbOiWGk64C4r95i6-T
3,GRANDMASTER,EjU2tbFog0rK4SqAAoCFE9m0sv0W1sT4_KV7gRWNc9lYnsQ,Shadow Nia,659,567,521,False,False,False,oKfzmIPK9dwtBnfiWjmNfRQeHQ5ER1V5WjAdipFbdd6D
4,GRANDMASTER,nh9JoV-evMo_8W-vFDrmRIcwHEeBToGFHJasCR85Ghl8xHWt,민식박고양이 휴,587,232,193,False,False,False,QZcv2KfqXHLi7ObJDe8Kh5JkjRcwPpHLrnW-NmvOHDwT0f...
